<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Modification of the fuselage length of an aircraft.

This notebook is based on the 3rd FAST-OAD-GA tutorial written by Aurélien. It aims to compare a reference architecture represented by the .XML file of an aircraft (Beechcraft 76, Cirrus 22, TBM-900) with a modified configuration. The modified architecture is produced by the FAST-OAD-GA module called modify_config which creates the corresponding .XML file. Then the .xml files of the reference aircraft and of the modified one go through all of the FAST-OAD-GA modules, with the generate_block_analysis function which calls each module. The variable viewer is called for both aircrafts after each module.

The modification implemented in this notebook is the increase of the fuselage length in order to add a row of passengers. The user has to define the multiplier of the span, and can also play on other parameters such as setting the engine position along the span.

The fastga code is from the root analysis_mode on GitHub.

Like in the tutorials, the structure of the notebook is based on data transfers between the data folder which stocks the .xml of the reference aircraft, and the workdir folder which stocks the .xml files passing through the modules of FAST-OAD-GA. After all the modules have been executed the final .xml files are placed in the output folder.

As the TBM-900 turbopropeller is still being modelized in FAST-OAD-GA, the Beechcraft 76 will be used for this analysis. There is no limit in the code for the increase in span, but some modules could have problems running with an extreme value (such as two times the original span).

## 1. Initialization of the work environment

### Choice of the aircraft

In [6]:
from ipywidgets import widgets, Layout
from fastoad.io import VariableIO
from IPython.display import display, clear_output

liste = widgets.Dropdown(
    options=[('Beechcraft Duchess 76', 'output_mda_beech.xml'), ('Cirrus SR22', 'output_mda_cirrus_sr22.xml'), ('TBM-900', 'output_mda_tbm900.xml')],
    value='output_mda_beech.xml',
    description='Reference Aircraft:',
    style={'description_width': 'initial'},
    layout=Layout(width='30%'),
)

# Initialization of the reference aircraft
ref_aircraft = 'output_mda_beech.xml'
print('The reference aircraft .xml file is '+ ref_aircraft)

def eventhandler(change):
    if (change.new):
        ref_aircraft = liste.value
        print('The reference aircraft .xml file is '+ ref_aircraft)

liste.observe(eventhandler, names='value')

display(liste)

The reference aircraft .xml file is output_mda_beech.xml


Dropdown(description='Reference Aircraft:', layout=Layout(width='30%'), options=(('Beechcraft Duchess 76', 'ou…

The reference aircraft .xml file is output_mda_cirrus_sr22.xml
The reference aircraft .xml file is output_mda_beech.xml


### Choice of the engine model 

In [7]:
liste = widgets.Dropdown(
    options=[('Basic IC Engine', "fastga.wrapper.propulsion.basicIC_engine"), ('Basic Turboprop', "fastga.wrapper.propulsion.basic_turboprop")],
    value="fastga.wrapper.propulsion.basicIC_engine",
    description='Propulsion Model:',
    style={'description_width': 'initial'},
    layout=Layout(width='30%'),
)

# Initialization of the engine model
engine_id = "fastga.wrapper.propulsion.basicIC_engine"
print('The engine model id chosen is '+ engine_id)

def eventhandler(change):
    if (change.new):
        engine_id = liste.value
        print('The engine model id chosen is '+ engine_id)

liste.observe(eventhandler, names='value')

display(liste)

The engine model id chosen is fastga.wrapper.propulsion.basicIC_engine


Dropdown(description='Propulsion Model:', layout=Layout(width='30%'), options=(('Basic IC Engine', 'fastga.wra…

### Preparation of files

In [11]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import logging
from fastoad.gui import VariableViewer
import shutil

# Define relative path.
current_path = os.getcwd()
DATA_FOLDER_PATH = pth.join(current_path, 'data')
WORK_FOLDER_PATH = pth.join(current_path, 'workdir')
OUTPUT_FOLDER_PATH = pth.join(current_path, 'output')

# Clear work folder.
shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)
os.mkdir(WORK_FOLDER_PATH)

# For using more screen width.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Final file names. These .xml files will be overwritten each time they pass through a module.
AIRCRAFT_REF_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_reference.xml')
AIRCRAFT_MOD_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_mod.xml')

# Copy the reference aircraft .xml file.
shutil.copy(pth.join(OUTPUT_FOLDER_PATH, ref_aircraft), AIRCRAFT_REF_FILE)
shutil.copy(AIRCRAFT_REF_FILE, AIRCRAFT_MOD_FILE)

# Definition of the files for the weight MDA.
CONFIG_FILE = pth.join(DATA_FOLDER_PATH, 'weight_loop.yml')
WORK_CONFIG_FILE = pth.join(WORK_FOLDER_PATH, 'weight_loop.yml')
shutil.copy(CONFIG_FILE, WORK_CONFIG_FILE)
WEIGHT_LOOP_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs_weight_loop.xml')

## 2. Definition of the modifications from the reference architecture

Describes the different parameters the user has to choose in order to obtain the desired modified configuration. The function generate_block_analysis is then used twice to create the .xml file related to the modified configuration.

The same parameters have to be used for the ComputeConfigMod class and for the UpdateXML class.

In [12]:
# Fuselage modifications
added_length = 0    # If 0, the length of a row of seats will be taken
added_section_x_ratio_front = 1
added_section_x_ratio_rear = 0
added_pax = 2     # Integer stating the number of passengers added in the design mission. It will be added to the npax_design of the xml, so take care not to exceed the new npax_max.
added_luggage = 10     # float stating the mass of luggage added in the design mission

fuselage_mod_parameters = [added_length, added_section_x_ratio_front, added_section_x_ratio_rear, added_pax, added_luggage]

In [13]:
from fastga.models.modify_config import ComputeConfigMod


compute_modify = api_cs23.generate_block_analysis(
        ComputeConfigMod(fuselage_mod=fuselage_mod_parameters),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )

output_mod = compute_modify({})

# Open viewer
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

In [ ]:
from fastga.models.modify_config.update_XML import UpdateXML


compute_update = api_cs23.generate_block_analysis(
        UpdateXML(fuselage_mod=fuselage_mod_parameters),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )

output_mod = compute_update({})

# Open viewer
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

## 3. Computation of the FASTGA modules

In [ ]:
from fastga.models.geometry import GeometryFixedTailDistance as Geometry
from fastga.models.aerodynamics.aerodynamics import Aerodynamics
from fastga.models.weight.weight import Weight
from fastga.models.weight.mass_breakdown.update_mtow import UpdateMTOW
from fastga.models.performances.mission import Mission
from fastga.models.load_analysis.loads import Loads
from fastga.models.handling_qualities.handling_qualities import ComputeHandlingQualities
from fastga.models.aerodynamics.external.openvsp.compute_vn import ComputeVNopenvsp


# GEOMETRY
compute_geometry_mod = api_cs23.generate_block_analysis(
        Geometry(propulsion_id=engine_id),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_geometry_mod({})

# AERODYNAMICS
compute_aero_mod = api_cs23.generate_block_analysis(
        Aerodynamics(
            propulsion_id=engine_id,
            use_openvsp=True,
            compute_mach_interpolation=True,
            compute_slipstream_cruise=True,
#             wing_airfoil_file="naca43013_3.af"
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_aero_mod({})

# WEIGHT / MTOW / PERFORMANCES loop

shutil.copy(AIRCRAFT_MOD_FILE, WEIGHT_LOOP_FILE)

eval_problem = api_cs25.evaluate_problem(WORK_CONFIG_FILE, overwrite=True)

OUTPUT_FILE = pth.join(WORK_FOLDER_PATH,'problem_outputs_weight_loop.xml')
shutil.copy(OUTPUT_FILE, AIRCRAFT_MOD_FILE)

# LOAD ANALYSIS
compute_loads_mod = api_cs23.generate_block_analysis(
        Loads(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_loads_mod({})

# HANDLING QUALITIES
compute_hq_mod = api_cs23.generate_block_analysis(
        ComputeHandlingQualities(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_hq_mod({})

# VN
compute_vn = api_cs23.generate_block_analysis(
        ComputeVNopenvsp(),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output = compute_vn({})

In [ ]:
from fastga.utils.postprocessing.analysis_and_plots import aircraft_geometry_plot

fig = aircraft_geometry_plot(AIRCRAFT_REF_FILE, name='reference aircraft', plot_nacelle = False)
fig = aircraft_geometry_plot(AIRCRAFT_MOD_FILE, name='modified aircraft', fig=fig, plot_nacelle = False)
fig.show()

## Save files for post processing

In [ ]:
OUTPUT_REF_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_ref_fuselage_be76.xml')
OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mod_fuselage_be76.xml')
shutil.copy(AIRCRAFT_REF_FILE, OUTPUT_REF_FILE)
shutil.copy(AIRCRAFT_MOD_FILE, OUTPUT_MOD_FILE)